In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import pandas as pd
import seaborn as sns
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.init as init
from torch import optim
import torch.nn.functional as F
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Data Sets
1. AT&T faces (Modified)
2. Omniglot (Single language - Gujarati)

Datasets

In [ ]:
dataset = 1
# Load the training dataset
if dataset == 1:
    folder_dataset = datasets.ImageFolder(root=r"data\faces\training")
    folder_dataset_test = datasets.ImageFolder(root=r"data\faces\testing/")
    print(f"Dataset: AT&T faces (Modified)\n")
elif dataset == 2:
    folder_dataset = datasets.ImageFolder(root=r"data\Gujarati\training")
    folder_dataset_test = datasets.ImageFolder(root=r"data\Gujarati\testing/")
    print(f"Dataset: Omniglot (Single language - Gujarati)\n")

d_no_epochs = 50
d_batch_size = 30
d_margin = 2.0
d_latent = 64
print(f"Summary of Training Folder:\n {folder_dataset}\n")
print(f"Summary of Testing Folder:\n {folder_dataset_test}")

# Resize the images and transform to tensors
transformation = transforms.Compose([transforms.Resize((100,100)),
                                        transforms.ToTensor()
                                        ])

In [ ]:
# Creating some helper functions
def imshow(img, text=None):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
        
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

Siamese Neural Network: CNN + FC

In [ ]:
#create the Siamese Neural Network
class SiameseNetwork(nn.Module):

    def __init__(self,latent):
        super(SiameseNetwork, self).__init__()
        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11,stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
            
            nn.Conv2d(96, 256, kernel_size=5, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(256, 384, kernel_size=3,stride=1),
            nn.ReLU(inplace=True)
        )

        # Setting up the Fully Connected Layers
        self.fc1 = nn.Sequential(
            nn.Linear(384, 1024),
            nn.ReLU(inplace=True),
            
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            
            nn.Linear(256,latent)
        )

    def forward_once(self, x):
        # This function will be called for each images
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, anchor, positive, negative):
        # In this function we pass in given images and obtain laten space embeddings
        output_anchor = self.forward_once(anchor)
        output_positive = self.forward_once(positive)
        output_negative = self.forward_once(negative)

        return output_anchor, output_positive, output_negative

Dataset loader

In [ ]:
class SiameseNetworkDataset(Dataset):
    def __init__(self,imageFolderDataset,transform=None,test=False):
        self.imageFolderDataset = imageFolderDataset
        self.transform = transform
        self.test = test
        self.files = iter(self.imageFolderDataset.imgs)
        self.firstexec = True
        self.current_file = next(self.files)

    def __getitem__(self,index):
        if not self.test:
            anchor_tuple = random.choice(self.imageFolderDataset.imgs)
            while True:
                #Look untill the same class image is found
                positive_tuple = random.choice(self.imageFolderDataset.imgs) 
                if (anchor_tuple[1] == positive_tuple[1]) and (anchor_tuple[0] != positive_tuple[0]):
                    break


            while True:
                #Look untill a different class image is found
                negative_tuple = random.choice(self.imageFolderDataset.imgs) 
                if anchor_tuple[1] != negative_tuple[1]:
                    break
            anchor = Image.open(anchor_tuple[0])
            positive = Image.open(positive_tuple[0])
            negative = Image.open(negative_tuple[0])
            label = [anchor_tuple[0].split('\\')[-2]]
        else:
            if self.firstexec==True:
                self.anchor_tuple = self.current_file              
                self.current_file = next(self.files)
                self.positive_tuple1 = self.current_file              
                self.current_file = next(self.files)
                self.firstexec = False
                
            if self.anchor_tuple[1] == self.current_file[1]:
                positive_tuple2 = self.current_file
                self.current_file=next(self.files)
            else:
                self.anchor_tuple = self.current_file              
                self.current_file = next(self.files)
                self.positive_tuple1 = self.current_file              
                self.current_file = next(self.files)
                positive_tuple2 = self.current_file

            
            anchor= Image.open(self.anchor_tuple[0])
            positive= Image.open(self.positive_tuple1[0])
            negative= Image.open(positive_tuple2[0])
            label = [self.anchor_tuple[0].split('\\')[-2]]

        anchor = anchor.convert("L")
        positive = positive.convert("L")
        negative = negative.convert("L")

        if self.transform is not None:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)
        
        
        return anchor, positive, negative, label
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)
    
    def get_current_value(self):
        if self.index >= len(self.iterable):
            raise StopIteration
        return self.iterable[self.index]

Sample batch visualization

In [ ]:
# Initialize the network
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset, transform=transformation,test=False)
# Create a simple dataloader just for simple visualization
vis_dataloader = DataLoader(siamese_dataset, shuffle=True, batch_size=15)

# Extract one batch
example_batch = next(iter(vis_dataloader))
# Example batch is a list containing 3x11 images
concatenated = torch.cat((example_batch[0], example_batch[1], example_batch[2]),0)
imshow(torchvision.utils.make_grid(concatenated,nrow=15))
print(f"Anchor labels: {example_batch[3]}")


In [ ]:
# Define the Triplet Loss Function
class TripletLoss(torch.nn.Module):
    def __init__(self, margin =d_margin):
        super(TripletLoss, self).__init__()
        self.margin = margin
    
    def forward(self, anchor, positive, negative):
        #Calculate the eucidian distance and calculate the Triplet Loss
        distance_positive = F.pairwise_distance(anchor, positive, keepdim=True)
        distance_negative = F.pairwise_distance(anchor, negative, keepdim=True)

        loss_triplet = torch.mean(F.relu(distance_positive - distance_negative + self.margin))

        return loss_triplet

In [ ]:
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset, transform=transformation, test=False)
train_dataloader = DataLoader(siamese_dataset, shuffle=True, batch_size=d_batch_size)
net = SiameseNetwork(d_latent).to(device)
criterion = TripletLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.0003)

In [ ]:
def test_accruracy(net):
    train_results = []
    labels = []
    net.eval()
    siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test, transform=transformation)
    # Create a simple dataloader just for simple visualization
    vis_dataloader = DataLoader(siamese_dataset, shuffle=True, batch_size=d_batch_size)
    running_loss_test = []
    loss_history_test = []
    with torch.no_grad():
        for epoch in range(0,1):
            # Initialize the running loss
            # Iterate over the batches in the dataloader
            for i, (anchor_test, positive_test, negative_test, _) in enumerate(vis_dataloader):
                # Transfer images and labels to the device
                anchor_test, positive_test, negative_test = anchor_test.to(device), positive_test.to(device), negative_test.to(device)
    
                # Forward pass
                output_anchor_test, output_positive_test, output_negative_test = [net.forward_once(anchor_test), net.forward_once(positive_test), net.forward_once(negative_test)]
                # Calculate the triplet loss
                loss_triplet_test = criterion(output_anchor_test, output_positive_test, output_negative_test)
    
                # Update the running loss
                running_loss_test.append(loss_triplet_test.item())

        return(np.sum(running_loss_test)/len(running_loss_test))


In [ ]:
counter = []
loss_history = [] 
iteration_no = 0
# Training loop
for epoch in range(d_no_epochs):
    # Initialize the running loss
    running_loss = 0.0

    # Initialize the start time for the epoch
    epoch_start_time = time.time()
    
    # Iterate over the batches in the dataloader
    for i, (anchor, positive, negative,_)  in enumerate(train_dataloader):
        # print(anchor,positive, negative, edgecase)
        """ """ # Transfer images and labels to the device
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        output_anchor, output_positive, output_negative = net(anchor, positive, negative)
        
        # Calculate the triplet loss
        loss_triplet = criterion(output_anchor, output_positive, output_negative)
        
        # Backward pass and optimization
        loss_triplet.backward()
        optimizer.step()

        # Update the running loss
        running_loss += loss_triplet.item()

        # Print the loss every 10 batches
        if (i + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{d_no_epochs}] \n\tBatch [{i+1}/{len(train_dataloader)}] \n\tLoss: {running_loss / 10}")
            running_loss = 0.0
        
        iteration_no += 1
        
    loss_history.append(loss_triplet.item())
    counter.append(iteration_no)
    # End of epoch
    # Calculate the epoch running time
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_start_time

    # Print epoch summary
    print(f"\tTime: {epoch_time:.2f} seconds")

In [ ]:
# Training complete
plt.plot(counter, loss_history)
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss over Epochs')
plt.grid(True)
plt.show()
print("Training finished.")

""" # Training complete
plt.plot(test_history)
plt.xlabel('Epoch')
plt.ylabel('Testing Loss')
plt.title('Testing Loss over Epochs')
plt.grid(True)
plt.show()
print("Training finished.") """


In [ ]:
# Load it into the SiameseNetworkDataset

siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=transformation)
test_dataloader = DataLoader(siamese_dataset, batch_size=1, shuffle=True)

# Grab one image that we are going to test
dataiter = iter(test_dataloader)
anchor, positive, negative, _ = next(dataiter)

for i in range(10):
    # Iterate over 10 images and test them with the first image (x0)
    anchor, positive, negative, _ = next(dataiter)

    # Concatenate the two images together
    concatenated = torch.cat((anchor, positive, negative), 0)
    
    output_anchor, output_positive, output_negative = net(anchor.to(device), positive.to(device), negative.to(device))
    euclidean_distance_positive = F.pairwise_distance(output_anchor, output_positive)
    euclidean_distance_negative = F.pairwise_distance(output_anchor, output_negative)
    imshow(torchvision.utils.make_grid(concatenated), f'Intracluster distance: {euclidean_distance_positive.item():.2f} -- Intercluster distance: {euclidean_distance_negative.item():.2f}')
print(output_anchor)

Save Parameters


In [ ]:
#torch.save({"model_state_dict": net.state_dict(),"optimizer_state_dict": optimizer.state_dict()}, "trained_model.pth")

Visualization of Latent space

In [ ]:
# Load it into the SiameseNetworkDataset

siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset, transform=transformation)
train_dataloader = DataLoader(siamese_dataset, batch_size=1, shuffle=True)

# Grab one image that we are going to test
dataiter = iter(train_dataloader)
anchor, positive, negative, _ = next(dataiter)
pos_dist = []
neg_dist = []
print(len(train_dataloader))
for i in range(len(train_dataloader)-1):
    # Iterate over 10 images and test them with the first image (x0)
    anchor, positive, negative, _ = next(dataiter)

    output_anchor, output_positive, output_negative = net.forward_once(anchor.to(device)), net.forward_once(positive.to(device)), net.forward_once(negative.to(device))
    euclidean_distance_positive = F.pairwise_distance(output_anchor, output_positive)
    euclidean_distance_negative = F.pairwise_distance(output_anchor, output_negative)
    # print(euclidean_distance_positive)
    pos_dist.append(euclidean_distance_positive.detach().cpu().numpy())
    neg_dist.append(euclidean_distance_negative.detach().cpu().numpy())

pos_dist = [x[0] for x in pos_dist]
neg_dist = [x[0] for x in neg_dist]

# Combine positive and negative distances for the box plot
distances = [pos_dist, neg_dist]

distance_data = {'Pos Dist':pos_dist,'Neg Dist':neg_dist}
df_train = pd.DataFrame(distance_data)
df_train.head()

# Set the style of the plot
sns.set(style="whitegrid")

# Create the box plot using seaborn
plt.figure(figsize=(15,5))
sns.boxplot(data=df_train,orient='h')

# Set the labels for x-axis and y-axis
plt.xlabel('Distance Type')
plt.ylabel('Distance')

# Set the title of the plot
plt.title('Box Plot of Positive and Negative Distances')

df_train.describe()

In [ ]:
# Load it into the SiameseNetworkDataset

siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test, transform=transformation,test = True)
test_dataloader = DataLoader(siamese_dataset, batch_size=1, shuffle=True)

# Grab one image that we are going to test
net.eval()
dist_positive_test = []
print(len(test_dataloader))
dataiter = iter(test_dataloader)
with torch.no_grad():
    while True:
        try:
            anchor, positive, positive_test, _ = next(dataiter)

            output_anchor = net.forward_once(anchor.to(device))
            output_positive_test = net.forward_once(positive_test.to(device))
            distance_anchor_positive_test = F.pairwise_distance(output_anchor, output_positive_test)
            dist_positive_test.append(distance_anchor_positive_test.detach().cpu().numpy())
        except StopIteration:
            break


dist_positive_test = [x[0] for x in dist_positive_test]

# Combine positive and negative distances for the box plot
distances = [dist_positive_test]

distance_data = {'Pos Test Dist' :dist_positive_test}
df_test = pd.DataFrame(distance_data)
df_test.head()
print(f"Threshold: {df_train['Pos Dist'].quantile(0.75)}")
print(len(df_test[df_test['Pos Test Dist'] < df_train['Pos Dist'].quantile(0.75)])/len(df_test))

df_train.describe()